### Global Peace Index vs Advisory Level


In [3]:
import pandas as pd
import plotly.express as px

# GPI
gpi = pd.read_csv('../../../data/processed/Global_Peace_Index_Processed.csv')
gpi_long = gpi.melt(id_vars="country", var_name="year", value_name="gpi_score")
gpi_long['year'] = gpi_long['year'].str.extract(r'(\d{4})').astype(int)

# GTI
gti = pd.read_csv('../../../data/processed/Global_Terrorism_Index_Processed.csv')
gti_long = gti.melt(id_vars="country", var_name="year", value_name="gti_score")
gti_long['year'] = gti_long['year'].str.extract(r'(\d{4})').astype(int)

# travel advisories
advisory = pd.read_csv('../../../data/processed/Travel_Advisories_Processed.csv')
advisory.columns = advisory.columns.str.lower()
advisory['country'] = advisory['country'].str.strip()
advisory_latest = advisory.sort_values('date updated').drop_duplicates(subset='country', keep='last')
advisory_latest = advisory_latest[['country', 'advisory level number']].rename(columns={'advisory level number': 'advisory_level'})

# merge
merged = gpi_long.merge(gti_long, on=["country", "year"], how="outer")
merged = merged.merge(advisory_latest, on="country", how="left")
merged_clean = merged.dropna(subset=["advisory_level"])
merged_clean = merged_clean[merged_clean['advisory_level'] == 4.0]

In [7]:
merged_clean

,country,year,gpi_score,gti_score,advisory_level
0,Afghanistan,2008,3.218,NaN,4.0
1,Afghanistan,2009,3.296,NaN,4.0
2,Afghanistan,2010,3.201,NaN,4.0
3,Afghanistan,2011,3.212,8.784,4.0
4,Afghanistan,2012,3.265,8.825,4.0
...,...,...,...,...,...
2732,Yemen,2020,3.406,5.333,4.0
2733,Yemen,2021,3.515,5.022,4.0
2734,Yemen,2022,3.595,4.708,4.0
2735,Yemen,2023,3.322,4.955,4.0


In [6]:
fig_gpi = px.scatter(
    merged_clean,
    x="gpi_score",
    y="advisory_level",
    animation_frame="year",
    hover_name="country",
    title="Global Peace Index vs Travel Advisory Level",
    labels={"gpi_score": "GPI Score", "advisory_level": "Travel Advisory Level"}
)

fig_gpi.update_layout(yaxis=dict(tickmode='array', tickvals=[1, 2, 3, 4]))
fig_gpi.show()

### Global Terrorism Index vs Advisory Level


In [8]:
gti_filtered = merged_clean.dropna(subset=["gti_score"])

fig_gti = px.scatter(
    gti_filtered,
    x="gti_score",
    y="advisory_level",
    animation_frame="year",
    hover_name="country",
    title="Global Terrorism Index vs Travel Advisory Level",
    labels={"gti_score": "GTI Score", "advisory_level": "Travel Advisory Level"}
)

fig_gti.update_layout(yaxis=dict(tickmode='array', tickvals=[1, 2, 3, 4]))
fig_gti.show()